In [44]:
import os
import pprint

import csv

# # import pandas as pd
from dotenv import load_dotenv
# from linkml_runtime import SchemaView
# from linkml_runtime.linkml_model.meta import ClassDefinition
from pymongo import MongoClient

import sqlite3

get a semsql database of envo DONE

mixs environmental triad slots require a term.id in curie notation (no underscores)



In [45]:
mongo_db_name = "nmdc"
output_file = '../local/env_local_scale_counts.tsv'
dot_env_file = "../local/.env"

In [46]:
def save_list_of_dicts_to_tsv(data, filename):
    # Extract the keys from the first dictionary in the list
    keys = data[0].keys()

    # Open the file in write mode with 'newline' parameter set to ''
    with open(filename, 'w', newline='') as tsvfile:
        writer = csv.DictWriter(tsvfile, fieldnames=keys, delimiter='\t')

        # Write the header
        writer.writeheader()

        # Write the rows
        writer.writerows(data)

In [47]:
load_dotenv(dot_env_file)

True

In [48]:
mongo_pw = os.getenv('SOURCE_MONGO_PASS')
# print(mongo_pw)
mongo_user = os.getenv('SOURCE_MONGO_USER')
print(mongo_user)

mam


In [49]:
client = MongoClient("localhost",
                     port=27777,
                     username=mongo_user,
                     password=mongo_pw,
                     authSource="admin",
                     authMechanism='SCRAM-SHA-256',  # todo should be an option
                     directConnection=True
                     )

In [50]:
db = client[mongo_db_name]

In [51]:
collection = db['biosample_set']

# Query the collection to get the count of each unique env_local_scale value
pipeline = [
    {"$group": {"_id": "$env_local_scale.term.id", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

results = collection.aggregate(pipeline)

# Print the results
lod = []
for result in results:
    # print(result['_id'], result['count'])
    temp = {
        'env_local_scale': result['_id'],
        'count': result['count']
    }
    lod.append(temp)

pprint.pprint(lod)



[{'count': 385, 'env_local_scale': 'ENVO:00002131'},
 {'count': 327, 'env_local_scale': 'ENVO:00000022'},
 {'count': 199, 'env_local_scale': 'ENVO:01001057'},
 {'count': 133, 'env_local_scale': 'ENVO:01001785'},
 {'count': 125, 'env_local_scale': 'ENVO_00000022'},
 {'count': 118, 'env_local_scale': 'ENVO:00005801'},
 {'count': 117, 'env_local_scale': 'ENVO:00000086'},
 {'count': 113, 'env_local_scale': 'ENVO:00000011'},
 {'count': 105, 'env_local_scale': 'ENVO:01001803'},
 {'count': 95, 'env_local_scale': 'UBERON:0001555'},
 {'count': 93, 'env_local_scale': 'ENVO:00000021'},
 {'count': 83, 'env_local_scale': 'ENVO:00000044'},
 {'count': 69, 'env_local_scale': 'ENVO:00000182'},
 {'count': 60, 'env_local_scale': 'biosphere reserve [ENVO:00000376]'},
 {'count': 53, 'env_local_scale': 'ENVO:00000292'},
 {'count': 38, 'env_local_scale': 'ENVO:00000444'},
 {'count': 34, 'env_local_scale': 'ENVO:00000260'},
 {'count': 33, 'env_local_scale': 'ENVO:00000384'},
 {'count': 31, 'env_local_scale': 

In [53]:
# Connect to the SQLite database file
conn = sqlite3.connect('../local/envo.db')

In [54]:
# Create a cursor object to execute SQL queries
cursor = conn.cursor()

In [55]:
# Write your SQL query
query = '''
select
	DISTINCT
	subject ,
	value
from
	statements s
where
	predicate = "rdfs:label"
'''

# Execute the query
cursor.execute(query)

In [56]:
# Fetch all the rows returned by the query
rows = cursor.fetchall()

In [57]:
term_to_label = {}
# Process the rows
for row in rows:
    # # Access the values in each row
    term = row[0]
    value = row[1]
    term_to_label[term] = value


In [62]:
for els_dict in lod:
    pprint.pprint(els_dict)
    if els_dict['env_local_scale'] in term_to_label:
        pprint.pprint(els_dict)
        pprint.pprint(term_to_label[els_dict['env_local_scale']])
        if term_to_label[els_dict['env_local_scale']] is not None:
            els_dict['label'] = term_to_label[els_dict['env_local_scale']]



{'count': 385, 'env_local_scale': 'ENVO:00002131'}
{'count': 385, 'env_local_scale': 'ENVO:00002131'}
'epilimnion'
{'count': 327, 'env_local_scale': 'ENVO:00000022'}
{'count': 327, 'env_local_scale': 'ENVO:00000022'}
'river'
{'count': 199, 'env_local_scale': 'ENVO:01001057'}
{'count': 199, 'env_local_scale': 'ENVO:01001057'}
'environment associated with a plant part or small plant'
{'count': 133, 'env_local_scale': 'ENVO:01001785'}
{'count': 133, 'env_local_scale': 'ENVO:01001785'}
'land'
{'count': 125, 'env_local_scale': 'ENVO_00000022'}
{'count': 118, 'env_local_scale': 'ENVO:00005801'}
{'count': 118, 'env_local_scale': 'ENVO:00005801'}
'rhizosphere'
{'count': 117, 'env_local_scale': 'ENVO:00000086'}
{'count': 117, 'env_local_scale': 'ENVO:00000086'}
'plain'
{'count': 113, 'env_local_scale': 'ENVO:00000011'}
{'count': 113, 'env_local_scale': 'ENVO:00000011'}
'garden'
{'count': 105, 'env_local_scale': 'ENVO:01001803'}
{'count': 105, 'env_local_scale': 'ENVO:01001803'}
'tropical forest

In [63]:
pprint.pprint(lod)

[{'count': 385, 'env_local_scale': 'ENVO:00002131', 'label': 'epilimnion'},
 {'count': 327, 'env_local_scale': 'ENVO:00000022', 'label': 'river'},
 {'count': 199,
  'env_local_scale': 'ENVO:01001057',
  'label': 'environment associated with a plant part or small plant'},
 {'count': 133, 'env_local_scale': 'ENVO:01001785', 'label': 'land'},
 {'count': 125, 'env_local_scale': 'ENVO_00000022'},
 {'count': 118, 'env_local_scale': 'ENVO:00005801', 'label': 'rhizosphere'},
 {'count': 117, 'env_local_scale': 'ENVO:00000086', 'label': 'plain'},
 {'count': 113, 'env_local_scale': 'ENVO:00000011', 'label': 'garden'},
 {'count': 105, 'env_local_scale': 'ENVO:01001803', 'label': 'tropical forest'},
 {'count': 95, 'env_local_scale': 'UBERON:0001555', 'label': 'digestive tract'},
 {'count': 93, 'env_local_scale': 'ENVO:00000021', 'label': 'freshwater lake'},
 {'count': 83, 'env_local_scale': 'ENVO:00000044', 'label': 'peatland'},
 {'count': 69, 'env_local_scale': 'ENVO:00000182', 'label': 'plateau'}

In [64]:
save_list_of_dicts_to_tsv(lod, output_file)

In [ ]:
# Close the cursor and the connection
cursor.close()
conn.close()

In [ ]:
# Close the MongoDB connection
client.close()